# Pixel-Level Localization on MVTec AD

### *Run these cells only when in Google Colab*

In [ ]:

# Clone the repository
!git clone https://github.com/beerger/mad_seminar_ws23.git
# Move all content to the current directory
!mv ./mad_seminar_ws23/* ./
# Remove the empty directory
!rm -rf mad_seminar_ws23/

In [ ]:
# Install additional packages
!pip install pytorch_lightning --quiet
!pip install lpips

## Imports for Local-Net

In [ ]:
import pytorch_lightning as pl
import yaml
import torch
import matplotlib.pyplot as plt
import json
from google.colab import drive

from model.local_net import LocalNet
from model.model_utils import load_resnet_18_teacher_model
from model.student_training_module import StudentTrainingModule
from image_net_data_loader import ImageNetDataModule
from model.one_layer_decoder import OneLayerDecoder

# autoreload imported modules
%load_ext autoreload
%autoreload 2

## 1. Pre-training

Until next numbered step the following code blocks will be part of the training of Local-Net. This is refered to as the pre-training of the framework, since the Local-Nets parameters will be fixed during training of Global-Net and DAD-head. This consists of two major steps:

* **Distillation**: on ImageNet, where the teacher network is pretrained ResNet-18.
* **Fine-tuning**: on some certain category of MVTec AD

Pre-processing in accordance (*) to ResNet-18 documentation
@ https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html

(*) According to documentation it's first resized to 256x256 then
center cropped to 224x224. This step has been skipped, 
and it is instead resized directly to 224x224. 
The reason for this is because the input patch size to Local-Net is 33x33
and ResNet-18 has input size 224x224, 
meaning that resizing to 256x256 and then cropping to desired size (224x224)
would result in altering the original spatial relationships and scale of the features of the image.
This could potentially lead to a mismatch when comparing features extracted from the resized and 
cropped image by ResNet-18 with those extracted from the original 33x33 image by Local-Net.

Pre-processing the data for distillation consists of 4 major steps:

1. Load images from Image-Net
2. Resize images to 256 x 256 (done by most applications, and mentioned in Supplementary Material)
3. Extract 33 x 33 patches from each resized image
4. Create two separate transform pipelines
    * For Local-Net: Convert the 33x33 patches to PyTorch tensors and normalize them
    * For ResNet-18 (teacher model): Resize the 33x33 patches to 224x224, then convert to tensors and normalize

<span style="color:yellow"> **Note**: All pre-processing is done by *ImageNetDataModule* provided in *image_net_data_loader.py* </span>

## Load the config

In [ ]:
with open('./configs/local_net_distillation.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Reproducibility
pl.seed_everything(config['seed'])

## Load and visualize data

Mount current Colab session to Google Drive (training/val images are stored here)

In [ ]:
# Will provide you with an authentication link
drive.mount('/content/drive')

Copy data from Google Drive to Colab VM's local storage

In [ ]:
# Copy training and validation zips from Google Drive
# Takes ~ 15 min
!cp "/content/drive/MyDrive/AnomalyDetection/Datasets/ImageNet/train.zip" "/content/"
!cp "/content/drive/MyDrive/AnomalyDetection/Datasets/ImageNet/val.zip" "/content/"

# Unzip
# Takes ~ 7 min
!unzip "/content/train.zip" -d "/content/train"
!unzip "/content/val.zip" -d "/content/val"

# Delete the zip files to free up space
!rm "/content/train.zip"
!rm "/content/val.zip"

Create data module for ImageNet

In [ ]:
def load_image_paths(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

train_image_paths = load_image_paths('/content/drive/MyDrive/AnomalyDetection/Datasets/ImageNet/train_image_local_paths.json')
val_image_paths = load_image_paths('/content/drive/MyDrive/AnomalyDetection/Datasets/ImageNet/val_image_local_paths.json')

data_module = ImageNetDataModule(
    train_image_paths, 
    val_image_paths, 
    batch_size=config['batch_size'], 
    num_workers=4, 
    caching_strategy='none'
)

Plot patches for Local-Net and ResNet-18

In [ ]:
# make sure batch_size in data_module is equal to BATCH_SIZE

BATCH_SIZE=config['batch_size']

# Reverse the normalization process done by ImageNetDataModule
# to avoid the following error:
# WARNING:matplotlib.image:Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
def denormalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)
    tensor = tensor * std + mean  # Reverses the normalization in-place
    return tensor.clamp(0, 1)  # Ensures the pixel values are within [0, 1]


# Retrieve one batch of images
patch_local, patch_resnet = next(iter(data_module.train_dataloader()))

# Denormalize the patches for visualization
patch_local = denormalize(patch_local)
patch_resnet = denormalize(patch_resnet)

fig, ax = plt.subplots(2, BATCH_SIZE, figsize=(20, 8))  # 2 rows, BATCH_SIZE columns

# Plotting patch_local images in the first row
for i in range(BATCH_SIZE):
    # Permute the tensor to the format (H, W, C)
    image = patch_local[i].permute(1, 2, 0)

    # Display the image
    ax[0, i].imshow(image.cpu().numpy())
    ax[0, i].axis('off')

# Plotting patch_resnet images in the second row
for i in range(BATCH_SIZE):
    # Permute the tensor to the format (H, W, C)
    image = patch_resnet[i].permute(1, 2, 0)

    # Display the image
    ax[1, i].imshow(image.cpu().numpy())
    ax[1, i].axis('off')

plt.show()

## Set up tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

## Set up all models for distillation

In [ ]:
# Assuming 'device' is either 'cuda' if a GPU is available, otherwise 'cpu'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

local_net = LocalNet(config).to(device)
resnet_18 = load_resnet_18_teacher_model('resnet18-5c106cde.pth', device)
decoder = OneLayerDecoder(config['local_net_output_dimensions'], 
                          config['resnet_output_dimensions']).to(device)

student_train_module = StudentTrainingModule(
    config, 
    student_model=local_net, 
    teacher_model=resnet_18, 
    decoder=decoder, 
    mode='distillation'
)

## Calculate number of epochs

In [ ]:
# Given by paper is batch size of 64 for 50k iterations
# Need to calculate max_epochs
total_iterations = config['iterations']
batch_size = config['batch_size']
num_training_images = len(train_image_paths)
# Calculate max_epochs
max_epochs = total_iterations / (num_training_images / batch_size)
max_epochs = int(max_epochs) + (max_epochs % 1 > 0)  # round up if not an integer

## Crete callbacks for training

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

# Setup the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath="/content/drive/MyDrive/AnomalyDetection/LocalNet/Checkpoints",  # Path where checkpoints will be saved
    filename="{epoch}-{val_loss:.2f}",  # Filename template
    monitor="val_loss",  # Metric to monitor for saving
    every_n_epochs=1,  # Save every epoch
    save_weights_only=True  # If True, save only the model weights, not the full model
)

## Setup new trainer

In [ ]:

trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    callbacks=[checkpoint_callback],
    logger=[
        pl.loggers.TensorBoardLogger(save_dir='./')
    ]
)

## Setup trainer from checkpoint


In [ ]:
# Change <CHECK_POINT.ckpt> to a valid checkpoint file located in
# /content/drive/MyDrive/AnomalyDetection/LocalNet/Checkpoints/

trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    callbacks=[checkpoint_callback],
    resume_from_checkpoint="/content/drive/MyDrive/AnomalyDetection/LocalNet/Checkpoints/<CHECK_POINT.ckpt>",
    logger=[
        pl.loggers.TensorBoardLogger(save_dir='./')
    ]
)

## Run distillation

In [ ]:
trainer.fit(student_train_module, datamodule=data_module)

## Save distilled model

In [ ]:
torch.save(local_net.state_dict(), '/content/drive/MyDrive/AnomalyDetection/LocalNet/local_net_distilled.pth')